In [1]:
#importing basic lib
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
#importing neccessary libs
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [3]:
filename = 'color_survey_answers.csv'

In [4]:
colours = pd.DataFrame()

In [5]:
colours = pd.read_csv(filename , delimiter='\t')

In [6]:
colours.head()

,id,user_id,datestamp,r,g,b,colorname
0,1,1,1.267419e+09,72,100,175,pastel blue
1,2,1,1.267419e+09,204,177,246,faint violet
2,3,1,1.267419e+09,182,226,245,baby blue
3,4,1,1.267419e+09,130,64,234,purple
4,5,2,1.267419e+09,75,49,234,blue


In [7]:
colours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363127 entries, 0 to 3363126
Data columns (total 7 columns):
id           int64
user_id      int64
datestamp    float64
r            int64
g            int64
b            int64
colorname    object
dtypes: float64(1), int64(5), object(1)
memory usage: 179.6+ MB


In [8]:
#converting date from float to datetime
from datetime import datetime
colours['datestamp'] = pd.to_datetime(colours['datestamp'],unit='s')

In [9]:
#extracting new colours into another dataframe from original
new_colour = pd.DataFrame()

In [10]:
new_colour = colours[colours['colorname'] == 'impatiens juice'].copy()

In [11]:
new_colour = colours[colours['colorname'] == 'caribbean surf'].append(new_colour)

In [12]:
new_colour = colours[colours['colorname'] == 'medium tan leather'].append(new_colour)

In [13]:
new_colour = colours[colours['colorname'] == 'babypoo'].append(new_colour)

In [14]:
#increaing the occurance of new colours
new_colour = new_colour.append([new_colour]*10)

In [15]:
new_colour.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 185291 to 3170468
Data columns (total 7 columns):
id           88 non-null int64
user_id      88 non-null int64
datestamp    88 non-null datetime64[ns]
r            88 non-null int64
g            88 non-null int64
b            88 non-null int64
colorname    88 non-null object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 5.5+ KB


In [16]:
#filtering out unwanted colours and colours which have occurance of 10000 or less
colours = colours.groupby('colorname').filter(lambda x: len(x) > 10000)

In [17]:
#combining new colours to the filtered original dataset
colours = pd.concat([colours,new_colour],ignore_index=True)

In [18]:
colours['colorname'].value_counts()

green                 310064
blue                  284220
purple                245832
pink                  129286
brown                  74782
red                    68985
light blue             57381
teal                   56336
orange                 51449
light green            49073
yellow                 43494
magenta                43139
grey                   35664
sky blue               34043
violet                 32407
lime green             30918
light purple           29221
turquoise              25925
lavender               25447
cyan                   24421
tan                    23710
dark green             23624
dark blue              23116
aqua                   23036
forest green           19112
mauve                  18899
bright green           18578
gray                   18003
olive                  17339
dark purple            16653
maroon                 16174
lilac                  15539
royal blue             14875
salmon                 14330
black         

In [19]:
#shuffling the dataset to have equal distribution of each colour in the dataset
colours = colours.sample(frac = 1,random_state= 10).reset_index(drop = True)

In [20]:
#custom binary encoding
single_colour = ['green','red','blue','yellow','purple','tan','gray','pink','olive','lime']
colours_cbe = colours.copy()

for color in single_colour:
    colours_cbe[color+'_bin'] = np.where(colours['colorname'].str.contains(color), 1, 0)
    

In [21]:
colours_cbe.head()

,id,user_id,datestamp,r,g,b,colorname,green_bin,red_bin,blue_bin,yellow_bin,purple_bin,tan_bin,gray_bin,pink_bin,olive_bin,lime_bin
0,1777700,80744,2010-03-11 23:28:48,78,200,22,bright green,1,0,0,0,0,0,0,0,0,0
1,3103477,138277,2010-03-29 19:38:09,99,183,185,blue,0,0,1,0,0,0,0,0,0,0
2,2921540,130578,2010-03-26 15:00:48,209,255,130,lime,0,0,0,0,0,0,0,0,0,1
3,1694089,77149,2010-03-11 01:55:13,63,37,133,purple,0,0,0,0,1,0,0,0,0,0
4,1291722,59196,2010-03-07 19:02:58,215,82,84,salmon,0,0,0,0,0,0,0,0,0,0


In [22]:
#taking a subset of dataset
#colours_small = colours_cbe.sample(10000,random_state= 10)

In [23]:
#x is the input features - taking rgb values
train = colours_cbe.iloc[:,3:6].values
#y is the target - taking the new encoded color bins, dropping the coloumn colourname
target = colours_cbe.iloc[:,7:].values

In [24]:
#target shape
target.shape

(2089594, 10)

In [25]:
train_data, test_data,train_target, test_target = train_test_split(train,target,test_size=0.25, random_state=0)

In [26]:
#creating pipeline with PCA and classifier
#clf =  svm.SVC(gamma='scale', decision_function_shape='ovr')

In [27]:
#parameter dict
param_grid = [ {'max_depth': np.arange(2, 10, 2), 'min_samples_split': np.arange(2, 10, 2), 'max_leaf_nodes' : np.arange(2, 10, 2)}]

In [36]:
#defining grid
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5,n_jobs=-1, verbose=1)
grid.fit(train_data,train_target)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed: 16.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'max_depth': array([2, 4, 6, 8]), 'min_samples_split': array([2, 4, 6, 8]), 'max_leaf_nodes': array([2, 4, 6, 8])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [29]:
#predicting test data
pred = grid.predict(test_data)

In [30]:
#test score
score = grid.score(test_data,test_target)*100
print("Accuracy: %f" % (score))

Accuracy: 57.228479


In [73]:
#test data
test_data[190]

array([ 51, 169, 132], dtype=int64)

In [74]:
#predicting the colourname for the rgb values
rgb_value = np.array([51, 169, 132]).reshape(1, -1)
predict_encoded = grid.predict(rgb_value)
arg_index = np.argmax(predict_encoded)
predict_colour = colours_cbe[colours_cbe.index.isin([arg_index])]['colorname']
print('The predicted colour is:\n', predict_colour)
#c= b.tolist()
#idx = colours_cbe[colours_cbe.index.isin(c)]
#dx = c.iloc[c,:]


The predicted colour is:
 2    lime
Name: colorname, dtype: object
